# Bruco (総当たりコヒーレンス計算) チュートリアル

このノートブックでは `gwexpy` の `Bruco` ツールを使用して、ブラインドコヒーレンススキャン（総当たり解析）を行う方法を示します。
Brucoはターゲットチャンネルと補助チャンネルリスト間のコヒーレンスを計算し、ノイズの結合（カップリング）を特定します。

オリジナルの実装はこちら: [https://github.com/mikelovskij/bruco](https://github.com/mikelovskij/bruco)


In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from gwpy.timeseries import TimeSeries, TimeSeriesDict
from gwexpy.analysis import Bruco
from astropy import units as u



## 1. 擬似データの生成

ターゲットチャンネルにいくつかのノイズラインが含まれており、一部の補助チャンネルがそのノイズ源を共有しているシナリオをシミュレーションします。

*   **Target**: 白色雑音 + 30Hz ライン + 60Hz ライン
*   **Aux1**: 白色雑音 + 30Hz ライン (30Hz で強い相関が期待される)
*   **Aux2**: 白色雑音 + 60Hz ライン (60Hz で強い相関が期待される)
*   **Aux3**: 白色雑音のみ (相関は期待されない)


In [2]:
# Parameters
duration = 60  # seconds
sample_rate = 256 # Hz
t = np.linspace(0, duration, duration * sample_rate)

# Helper to create data
def create_data(name, freqs=[]):
    noise = np.random.normal(0, 0.1, size=len(t))
    signal = np.zeros_like(t)
    for f in freqs:
        signal += 0.5 * np.sin(2 * np.pi * f * t)
    return TimeSeries(noise + signal, t0=0, sample_rate=sample_rate*u.Hz, name=name)

# Create channels
target = create_data("H1:TARGET", freqs=[30, 60])
aux1 = create_data("H1:AUX1", freqs=[30])
aux2 = create_data("H1:AUX2", freqs=[60])
aux3 = create_data("H1:AUX3", freqs=[])

print("Data generated successfully.")


Data generated successfully.


## 2. Bruco の実行

ターゲットチャンネル名と補助チャンネルのリストを指定して `Bruco` を初期化します。

**注意**: 実際の環境では `Bruco` は `TimeSeries.get` を使用して NDS やフレームからデータを取得します。ここではメモリ上の生成データを使用するため、データ取得メソッドをモック（偽装）してローカルデータを返すようにします。


In [3]:
# Define channel names
target_channel = "H1:TARGET"
aux_channels = ["H1:AUX1", "H1:AUX2", "H1:AUX3"]

# Initialize Bruco
comp = Bruco(target_channel, aux_channels)

# Mocking the data fetching
def mock_get_target(channel, start, end, **kwargs):
    return target

def mock_get_aux(channels, start, end, **kwargs):
    # channels is a list
    data = {"H1:AUX1": aux1, "H1:AUX2": aux2, "H1:AUX3": aux3}
    return TimeSeriesDict({c: data[c] for c in channels})

# Apply patches to the module where TimeSeries is used
import gwexpy.analysis.bruco as bruco_module
orig_ts_get = bruco_module.TimeSeries.get
orig_tsd_get = bruco_module.TimeSeriesDict.get

bruco_module.TimeSeries.get = mock_get_target
bruco_module.TimeSeriesDict.get = mock_get_aux

try:
    print("Running Bruco scan...")
    # Run computation
    # fftlength=2.0 means 0.5Hz frequency resolution
    results = comp.compute(start=0, duration=duration, fftlength=2.0, overlap=1.0, nproc=1)
finally:
    # Always restore original methods
    bruco_module.TimeSeries.get = orig_ts_get
    bruco_module.TimeSeriesDict.get = orig_tsd_get


Running Bruco scan...


## 3. 結果の確認

結果の DataFrame は `max_coherence` (最大コヒーレンス) で降順ソートされています。
`H1:AUX1` と `H1:AUX2` が高いコヒーレンスを示し上位に、`H1:AUX3` が下位に来ることが期待されます。


In [5]:
# Display the results
results

,channel,max_coherence,mean_coherence,freq_at_max
0,H1:AUX2,0.999183,0.029491,60.0
1,H1:AUX1,0.999147,0.026804,30.0
2,H1:AUX3,0.096195,0.019635,6.5
